# Dipole Radiation

Any accelerating charge produces electromagnetic radiation.  If the charges in a neutral object — like an atom or a radio antenna — are undergoing periodic motion, we can approximate the system as an **oscillating electric dipole**.  As discussed in Chapter 9 of _CER_ or [Chapter 28 of the Feynman Lectures](https://www.feynmanlectures.caltech.edu/I_28.html), the __radiation field__ at point $\vec{r}$ in the $xz$-plane far from a dipole oscillating along the $z$-axis at the origin is

$$\vec{E}(\vec{r}, t) = \dfrac{1}{c^2}
\, \omega^2 p_0 \sin \theta
\, \dfrac{\cos \omega(t - r/c)}{r}
\, (\hat{e}_x \cos \theta - \hat{e}_z \sin \theta)$$

The goal of this exercise is to tame this expression and create a movie of an oscillating electric field.

First, the variables:

| Symbol | Meaning |
| --- | --- |
| $\omega$ | angular frequency of oscillation |
| $p_0$ | maximum dipole moment |
| $c$ | speed of light |
| $r$ | distance from origin to point of interest |
| $t$ | current time |
| $\theta$ | angle from $y$-axis to point of interest |

The final term in parentheses is a unit vector that gives the direction of the field.

### Warm Up

Our goal is to watch the electromagnetic waves propagate outward from the dipole as it oscillates.  This means we need to choose the time increments and spatial grid of points appropriately.  We need to ...

- Determine the duration of the simulation if we want to watch 10 complete cycles of oscillation.
    - 700 nm => 428 THz => 2.33 fs
    - 2.33 fs x 10 = 23.3 fs

- Determine the size of the grid if we want to see at least 5 wavelengths in each direction.
    - 700 nm x 5 = 3.5 um

- Determine a reasonable time increment `dt` and grid spacing `dx` for simulating red light with a wavelength of $700 \ {\tt nm}$.
    - 1% x 2.33 fs = 23.3 as
    - 1% x 700 nm = 7 nm

We do not have the computer hardware of lifespan to simulate a 1 billion by 1 billion grid of points for 10 billion time steps!

___Solution:___

- The wavelength of the light is $\lambda = 7.0 \, 10^{-7} \ {\tt m}$.

- The period of a wave is $T = \dfrac{1}{f} = \dfrac{\lambda}{c}$.  For red light, this works out to $T \approx 2.335 \cdot 10^{-15} \ {\tt s}$.

___Dimensional Analysis:___

These values are very small, and the frequency and speed of light are very large. We might run into roundoff error if we are not careful.  Instead, let's choose our units so that numerical values are on the order of 1.

First, note that we can rewrite the argument of the cosine function as follows:
$$\omega(t - R/c) = 2\pi \left(\dfrac{t}{T} - \dfrac{r}{\lambda}\right)$$
If we measure time in femtoseconds and distance in micrometers, all numbers in this expression will be of order 1.

Second, we can now rewrite the expression for the electric field as follows:
$$\vec{E}(\vec{r}, t)
= E_0 \, \dfrac{\lambda}{r} \, \cos 2\pi \left( \dfrac{t}{T} - \dfrac{r}{\lambda}\right)
\, (\hat{e}_x \cos \theta - \hat{e}_z \sin \theta)  \sin \theta
$$
where
$$E_0 = \dfrac{\omega^2 p_0}{\lambda c^2} = 4 \pi^2 \dfrac{p_0}{\lambda^3}$$

Different values of $E_0$ change the overall magnitude of the electric field, but not the relative size of the components at different points.  Thus, we can set $E_0 = 1$ during the calculation, and we can restore the units if we need to compute forces at another time.

This type of dimensional analysis can greatly simplify codiing and often leads to more accurate results because you do not "forget" a factor $10^9$ somewhere in the code.

Now, let's make a movie!

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os import system
from IPython.display import HTML, clear_output

## Linear Dipole

Our first simulation will illustrate the radiation field of a dipole oscillating along the z-axis.  This is much like the field created by a single radio tower.  Before you run the following cells, make some predictions.

- What do you expect the field to look like?
- In what directions will the strength of the field be strongest? Weakest?

***Write your predictions below.***

From doing HAM radio, I actually know what the field of a dipole should look like! Although that is generally graphed in terms of intestity around the antenna to determine what angle most of the power is being sent out at. That being said, for the electric and magentic field, I think the electric field will point radially from the dipole and grow and shrink with the change in current. I don't know if this field is being modeled with some potential or current density function on the y axis, but wherever the peaks of that function are will either be the maximum or minimum values of magnetic or electric field. The fields will be greatest closest to the antenna, and degrade as they get further away.

In [8]:
from scipy.constants import c as speed_of_light

# Wavelength in m.
lambda0 = 700e-9

# Wavelength in um.
wavelength = lambda0 * 1e6

# Period in fs.
period = 1e15 * lambda0 / speed_of_light

# Define grid of points and array of times for plotting.
t_max = period
dt = period / 64
t_values = np.arange(0, t_max + dt, dt)
N = len(t_values)

x_max = 3 * wavelength
dx = wavelength / 10
coordinates = np.arange(-x_max, x_max + dx, dx)

X,Z = np.meshgrid(coordinates, coordinates)
R = np.sqrt(X**2 + Z**2 + 1e-6)
theta = np.arctan2(X,Z)

# Define field strength and cutoff for rescaling large values.
E0 = 2.0
E_cut = 1.0
R_cut = 0.5*wavelength

# Remove any existing movie frames.
system('rm -f *-linear-dipole.jpg')

# It is essential that the frames be named in alphabetical order.
# {:03d} will display integers with three digits and insert zeros if needed:
# '000_movie.jpg', '001_movie.jpg', etc.
file_name = "{:03d}-linear-dipole.jpg"


plt.figure(figsize=(6,6), dpi=200)
# Generate frames and save each figure as a separate .jpg file.
for i, t in enumerate(t_values):
    # Compute magnitude of field.
#     ## Dipole oscillating along z-axis.
#     pz = np.cos(2*np.pi * (t/period))
#     px = 0
#     E =  -E0 * wavelength * np.sin(theta) * np.cos(2*np.pi * (t/period - R/wavelength)) / R
    
    # Dipole oscillating along x-axis.
    pz = 0
    px = np.cos(2*np.pi * (t/period))    
    E =  E0 * wavelength * np.cos(theta) * np.cos(2*np.pi * (t/period - R/wavelength)) / R

    # Use this for the oscillating dipole.
    Ex = E * np.cos(theta)
    Ez = -E * np.sin(theta)

    # Rescale large values.
    Ex[R<R_cut] = 0
    Ez[R<R_cut] = 0
    E = np.sqrt(Ex**2 + Ez**2) 
    bad = E > E_cut
    Ex[bad] = E_cut * Ex[bad] / E[bad]
    Ez[bad] = E_cut * Ez[bad] / E[bad]
    
    # Make plot and save to file.
    plt.quiver(X, Z, Ex, Ez, pivot='middle', lw=1, scale=40)
    plt.quiver(0,0,px,pz, pivot='middle', color='red', lw=1, scale=10)
    plt.xlabel('X [um]')
    plt.ylabel('Z [um]')
    plt.savefig(file_name.format(i))  # save current plot

    # Clear plot for next frame.
    plt.cla()
    
    print("Completed frame %d of %d.\r" % (i+1,N), end='')

plt.close()

In [9]:
!ffmpeg -y -i %03d-linear-dipole.jpg -pix_fmt yuv420p linear-movie.mp4

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 10.2.1 (GCC) 20200726
  configuration: --disable-static --enable-shared --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libsrt --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libgsm --enable-librav1e --disable-w32threads --enable-libmfx --enable-ffnvcodec --enable-cuda-llvm --enable-cuvid --enable-d3d11va 

In [10]:
# Play the movie within the notebook.
HTML("""
    <video width="800" height="800" alt="test" controls loop>
        <source src=%s type="video/mp4">
    </video>
""" % "linear-movie.mp4")

### Questions

Describe the field pattern.

- What does the field to look like?
- In what directions will the strength of the field be strongest? Weakest?

***Report your observations below.***

The charge density along the axis of the dipole will increase and decrease following some sinusoidal function. The chage density increases one way then changes direction and increases in the other direction. At the peak of this sinusoidal function, the electric field is storngest. At the point where the charge density in the dipole is 0, the field around the dipole is 0. This creates a wave of electric fields in different directions separated by regions of no fields. Along the axis of the dipole, there is a conical shape in which there is very little change in the electric field. This is due to the beamwidth of the electric field, which is the same for all halfwave dipoles. The electric field seems to decrease as it gets further from the antenna

Would you describe this radiation as a "plane wave"?  Why or why not?

***Share your opinion below.***

Although this component only describes the electric field, the magnetic field has the same magnitude but in the perpendicular direction of the electric field. I suppose if we were to actually look at the emitted wave in 3D, each "block" of electric field would appear like spherical shells. The sperical shells move away from the dipole with the same phase and frequency but degrading amplitude. Since nothing except the amplitude of this wave changes over the distance the wave travels in the plane of the dipole, we can say this wave is a plane wave.

**Different Axis**

How would you expect the pattern to change if you rotated the dipole by 90°?

***Report your prediction below.***

I predict that the pattern will look exactly the same, except with the "dead zones" (the areas with no electric field) will be in the x axis. Nothing else ahs changed about the dipole except its orientation, so it shoud not change the shape of the wave, just the direction.

The code above contains instructions for viewing a dipole oscillating along the x-axis.

Comment out the three lines below
```
    ## Dipole oscillating along z-axis.
```
Uncomment the three lines below
```
    ## Dipole oscillating along x-axis.
```
Rerun the simulation.

- Describe what you see.
- Assess your predictions.

***Report your observations below.***

My predictions were completely correct, the dipole shape only changed orientations. The entire wave, beamwidth, and electric field are the same through the medium but they have been flipped to propogate on the z axis. Not much else to say about it...

## Rotating Dipole

Instead of a single dipole oscillating along the $y$-axis, analyze a dipole that is rotating in the $xy$-plane.  The dipole moment is

$$\vec{p}(t) = p_0 \left(\hat{e}_x \cos \omega t + \hat{e}_z \sin \omega t \right)$$

This is equivalent to two _linear_ dipoles at right angles to each other, oscillating $90^\circ$ out of phase.

The code below simply adds together the fields of the two dipoles you just observed.  Before you run it, make some predictions.

- What do you expect the field to look like?
- In what directions will the strength of the field be strongest? Weakest?

***Write your predictions below.***

The "dead zone" now will rotate with the antenna, and therefore will have 0 field in spirals around the antenna. There will also be a secondary action due to the oscillation of the electric field of the dipole. This means the wave will increase and decrease in spirals, but then also have a constant spiral which has no electric field. I imagine the 3D field looking like a snail's shell, where the inside starts out small and then gets bigger towards the edges, with gaps between each section. 

In [145]:
from scipy.constants import c as speed_of_light

# Wavelength in m.
lambda0 = 700e-9

# Wavelength in um.
wavelength = lambda0 * 1e6

# Period in fs.
period = 1e15 * lambda0 / speed_of_light

# Define grid of points and array of times for plotting.
t_max = period
dt = period / 64
t_values = np.arange(0, t_max + dt, dt)
N = len(t_values)

x_max = 3 * wavelength
dx = wavelength / 10
coordinates = np.arange(-x_max, x_max + dx, dx)

X,Z = np.meshgrid(coordinates, coordinates)
R = np.sqrt(X**2 + Z**2 + 1e-6)
theta = np.arctan2(X,Z)

# Define field strength and cutoff for rescaling large values.
E0 = 2.0
E_cut = 1.0
R_cut = 0.5*wavelength

# Remove any existing movie frames.
system('rm -f *-rotating-dipole.jpg')

# It is essential that the frames be named in alphabetical order.
# {:03d} will display integers with three digits and insert zeros if needed:
# '000_movie.jpg', '001_movie.jpg', etc.
file_name = "{:03d}-rotating-dipole.jpg"


plt.figure(figsize=(6,6), dpi=200)
# Generate frames and save each figure as a separate .jpg file.
for i, t in enumerate(t_values):
    # Plot the dipole that is creating the fields.
    pz = -np.cos(2*np.pi * (t/period))
    px = -0.5*np.sin(2*np.pi * (t/period))

    # Compute magnitudes of fields.
    E1 = E0 * wavelength * np.sin(theta) * np.cos(2*np.pi * (t/period - R/wavelength)) / R
    E2 = -0.5 * E0 * wavelength * np.cos(theta) * np.sin(2*np.pi * (t/period - R/wavelength)) / R

    Ex = (E1 + E2) * np.cos(theta)
    Ez = -(E1 + E2) * np.sin(theta)

    # Rescale large values.
    Ex[R<R_cut] = 0
    Ez[R<R_cut] = 0
    E = -np.sqrt(Ex**2 + Ez**2) 
    bad = E > E_cut
    Ex[bad] = E_cut * Ex[bad] / E[bad]
    Ez[bad] = E_cut * Ez[bad] / E[bad]

    # Make plot and save to file.
    plt.quiver(X, Z, Ex, Ez, pivot='middle', lw=1, scale=40)
    plt.quiver(0,0,px,pz, pivot='middle', color='red', lw=1, scale=10)
    plt.xlabel('X [um]')
    plt.ylabel('Z [um]')
    plt.savefig(file_name.format(i))  # save current plot

    # Clear plot for next frame.
    plt.cla()
    
    print("Completed frame %d of %d.\r" % (i+1,N), end='')

plt.close()

In [146]:
!ffmpeg -y -i %03d-rotating-dipole.jpg -pix_fmt yuv420p rotating-movie.mp4

# display.clear_output()

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 10.2.1 (GCC) 20200726
  configuration: --disable-static --enable-shared --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libsrt --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libgsm --enable-librav1e --disable-w32threads --enable-libmfx --enable-ffnvcodec --enable-cuda-llvm --enable-cuvid --enable-d3d11va 

In [147]:
# Play the movie within the notebook.
HTML("""
    <video width="800" height="800" alt="test" controls loop>
        <source src=%s type="video/mp4">
    </video>
""" % "rotating-movie.mp4")

### Questions

Describe the field pattern.

- What does the field to look like?
- In what directions will the strength of the field be strongest? Weakest?

***Report your observations below.***

The field looks almost as my predictions stated. I realized there is no secondary action due to the diople increasing and decreasing in magnitude, only where the tips of the dipole are there is no field. This is what I said in the prediction, except I thought the magnitude of the gap would be larger, more like the cone shape of the first movie. I noticed that the field flipped depending on which side of the dipole was facing the outside of the graph. The field still died off near the edges of the field, but it starts out initally equal in all directions. 

Would you describe this radiation as a "plane wave"?  Why or why not?

***Share your opinion below.***

I would not describe this wave as a plane wave, it does not keep the same period in a rectangular plane. The distance between the waves spread out, meaning that the period does not stay the same. The waves even appear to slow down as they get further away from the source. Perhaps this is just an illusion caused by the amplitude decreasing with distance. 

Modify the code above to make the dipole rotate in the opposite direction ***and*** produce the correct radiation pattern.

**Hint:** It only requires two minus signs ...

- Describe your attempts and your results.
- Why are only two minus signs required?

***Report your observations below.***

Well... I got something! The first sign I changed was the pz and px signs. By switching the negative to the z component, I was able to get the dipole to start to rotate in the opposite direction. That was pretty simple, so by the same logic I switched the signs of the Ex and Ez components. This resulted in the dipole spinning the correct way but the field was wrong. Next, I tried changing different conbinations of the magnitudes and x and z components to see if I could get something that worked. By changing the Ez component to positive and changing theta to negative, I got the correct answer. I did end up only adding two minus signs (although I think I took away more). The field looks wierd, like it got more dense near the dipole. I don't know if this is an issue with the calculations or something wrong, but further away from the dipole the field looked the same

What would happen if the dipole along the x-axis were **half as large** as the dipole along the z-axis?

- How do you think this would change the field pattern?  Why?

***Report your predictions below.***

I think that if the dipole were half as large on the x axis, the rotation of the dipole would not be smooth around the circle. What I mean is that the x value will only allow a maximum of 1/2, while the z can still change between 1 and -1. This will give a shrinking and growing effect where when the dipole points along the x-axis, it will be half the size of the dipole when it points along the z axis.

Modify the code above so that the dipole along the  x-axis is **half as large** as that along the z-axis.  You will need to modify `px` and `E2`.

- How does this change the field pattern?

***Report your observations below.***

I noticed the circle around the dipole turned into more of an ellipse. That being said, after I tried to change some things around the notebook stopped working! I went back to Dr. Kinder's GitHub page to see if I could find the original code, but even after changing it back it doesn't seem to be working. Well the original still works but it won't let me set both the E2 and px value to be multiplied by 0.5.

After messing around with it for a bit, I found that I had set the Ex component to half the original value. Even then, now I don't see any change in the field when E2 is set to half the value. I really don't know what's going on here. I would have assumed that the first picture I saw with the ellipse field around the dipole would be correct, but shouldn't the ellipse spin with the dipole? I don't know, I would imagine you'd end up with two waves, one with a frequency of the spinnign dipole and one with the frequency of the wave cased by the change in the dipole. 

# Reflection and Summary

- What are the major takeaways of this assignment for you?
- What was the most difficult part of this assignment?
- What was the most interesting part of this assignment?
- What questions do you have?

***Include your response below.***

I really enjoyed this last notebook, I think that overall it illustrated what the electric field looks like around a radiating element. I wish there was a little better connection between the model and an actual antenna, since it's not like I can physically build a 700 nm halfwave dipole. I guess this would model somthing like an atom which has an electron continually jumping between energy levels. The part where we spin the dipole might represent a atom spinning while it moves. The chaning in the length of the dipole while it spins might represent contraction or stretching of bond lengths as it spins. These were all things I was thinking about, but I wonder how accurate that might be? How could this model be applied to something like flourescent chemicals or LEDs or anything else that produces light? Not only that, but what insights could we gain for designing much larger systems like radio all the way up to microwave antennas. The most interesting part of this assignment was playing around with the rotating antenna, but I couldn't get certain sections of that part to work. It was kind of frusterating, but at the same time I feel like I got a very good understanding of how the system worked in python. The most difficult part of this assignment was trying to get the graph to work with the half valued fields and dipole in the rotating section. I could not figure out why it worked the first time I tried it but after I changed the magnitude it stopped working! I've run into issues with Python where after changing one thing and then going back to the first code breaks it for some reason and I don't know if it's a problem with Jupyter or Colab or whether I've screwed up code that bad. Anyways, I didn't know how to fix it, so I got the first part where the dipole worked to go, and I got one look at the changing field but nothing after that. I really apologize, I hate to leave stuff like that but this has to get in.

As a more general point about the notebooks, I feel that they were very helpful, and I learned a lot about not only Python but also visualizing what fields look like. I really liked the Fenics stuff, as much as it challenged me (to put it in polite terms) and I feel like it was better than the book for describing things like vector and scalar potential with the heatmaps. I think my favorite notebooks were the magnetic field around an iron core and the waveguide one. Really great job with the class, I've learned more in this class than any class I think I've ever taken at this school. Both professors really care about the students and are extremely knowledgable, which made for an organized and open learning environment.

# Challenge Problems

The challenge problems below ask you to extend the simulation above.  

## Near Fields

The simulations above only include the radiation field — the portion of the field that decreases as $1/r$.  Far from the source, this is the only part that matters.  Near the source, the other contributions to the electric field are the most important.

Refer to Equation (9.39) in _CER_.  Try to plot the full electric field for the dipole

$$\vec{p} = p_0 \, \hat{e}_z \, \cos \omega t$$


## Multiple Sources

In the code above, all of the dipoles are at the origin.  Adapt the code above to simulate the field of two dipoles at different locations.


## 3D Plots

Adapt the code to plot the field of a linear dipole or a rotating dipole in three dimensions.


## Plot Everything

Add the magnetic field and the Poynting vector to a 3D simulation!

In [ ]:
### Replace with your code.

## A note on coordinates and formulae

The simulation above uses spherical coordinates to match equations in textbooks.  However, this is not always convenient for simulating problems on a grid.  For instance, the equations for the fields assume the dipoles are at the origin.  If there are two dipoles at different locations, "$\hat{e}_r$" and "$\hat{e}_\theta$" in the formulas are ***not*** the same vector for both dipoles.

It might be easier to work in Cartesian coordinates.  The following relations may be useful if you attempt the challenge problems. If the location of the dipole is $(x_0, y_0, z_0)$, then the textbook formulae translate as follows.

\begin{align*}
r &= \sqrt{(x-x_0)^2 + (y-y_0)^2 + (z-z_0)^2} \\
\cos \theta & = \dfrac{z-z_0}{r} \\
\sin \theta &= \dfrac{\sqrt{(x-x_0)^2 + (y-y_0)^2}}{r} \\
\cos \phi &= \dfrac{x-x_0}{\sqrt{(x-x_0)^2+(y-y_0)^2}} \\
\sin \phi &= \dfrac{y-y_0}{\sqrt{(x-x_0)^2+(y-y_0)^2}} \\
\hat{e}_r &= \hat{e}_x \, \sin \theta \cos \phi + \hat{e}_y \, \sin \theta \sin \phi + \hat{e}_z \, \cos \theta \\
\hat{e}_\theta &= \hat{e}_x \, \cos \theta \cos \phi + \hat{e}_y \, \cos \theta \sin \phi - \hat{e}_z \sin \theta \\
\hat{e}_\phi &= -\hat{e}_x \, \sin \phi + \hat{e}_y \, \cos \phi
\end{align*}